In [6]:
from database.adatabase import ADatabase
from extractor.alp_client_extractor import ALPClientExtractor
from extractor.tiingo_extractor import TiingoExtractor
from processor.processor import Processor as processor
from datetime import datetime, timedelta
from tqdm import tqdm 
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
from time import sleep
import matplotlib.pyplot as plt



In [21]:
start = datetime.now() - timedelta(days=2)
extractor = ALPClientExtractor(os.getenv("APCAKEY"),os.getenv("APCASECRET"))
tickers = ["AAVE", "AVAX", "BAT", "BCH", "BTC", "CRV", "DOGE", "DOT", "ETH", "GRT", "LINK", "LTC", "MKR", "SHIB", "SUSHI","USDT","USDC", "UNI", "XTZ", "YFI"]
tickers = [x +"/USD" for x in tickers]
prices = []
for ticker in tickers:
    response = extractor.crypto(ticker,start)
    price = pd.DataFrame(response["bars"][ticker]).rename(columns={"c":"adjclose","t":"date"})[["date","adjclose"]]
    price["date"] = pd.to_datetime(price["date"])
    price = price.sort_values("date")
    price["rolling_return"] = ((price["adjclose"].rolling(30).mean() - price["adjclose"]) / price["adjclose"])
    price["return"] = price["adjclose"].pct_change(5)
    price["avg_return"] = price["return"].rolling(30).mean()
    price["coev"] = (price["adjclose"].rolling(30).mean() / price["adjclose"].rolling(30).std())
    price["ticker"]= ticker
    prices.append(price)

prices = pd.concat(prices).sort_values("date").dropna()

In [23]:
metrics = [
    "rolling_return",
    "return",
    "avg_return"
    ,"coev"
]

In [38]:
date_range = list(prices["date"].unique())

cash = 100
base_position = {"date":date_range[0],"ticker":"","quantity":0,"adjclose":0,"buy_price":0,"buy_date":date_range[0]}
positions = []
analysis = []
for metric in tqdm(metrics):
    for ascending in [True,False]:
        position = base_position.copy()
        for date in date_range:
            try:
                position = position.copy()
                position["date"] = date
                today = prices[prices["date"]==date]
                opportunity = today.sort_values(metric,ascending=ascending).iloc[0]
                if today.index.size >= len(tickers):
                    if position["ticker"] != "":
                        position["adjclose"] = today[today["ticker"]==position["ticker"]]["adjclose"].iloc[0].item()
                        if (date - position["buy_date"]).days > float(5/24*60):
                            cash = position["adjclose"] * position["quantity"]
                            position = base_position.copy()
                            position["date"] = date
                            position["ticker"] = opportunity["ticker"]
                            position["quantity"] = cash * 0.9975 / opportunity["adjclose"] 
                            position["adjclose"] = opportunity["adjclose"]
                            position["buy_price"] = opportunity["adjclose"]
                            position["buy_date"] = date
                    else:
                            position["date"] = date
                            position["ticker"] = opportunity["ticker"]
                            position["quantity"] = cash * 0.9975 / opportunity["adjclose"] 
                            position["adjclose"] = opportunity["adjclose"]
                            position["buy_price"] = opportunity["adjclose"]
                            position["buy_date"] = date
                positions.append(position.copy())
            except Exception as e:
                print(str(e))
        position["metric"] = metric
        position["ascending"] = ascending
        position["pv"] = position["adjclose"] * position["quantity"]
        analysis.append(position)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:16<00:00,  4.20s/it]


In [39]:
a = pd.DataFrame(analysis)
a.sort_values("pv",ascending=False)

,date,ticker,quantity,adjclose,buy_price,buy_date,metric,ascending,pv
1,2024-09-25 19:38:00+00:00,GRT/USD,573.547803,0.180885,0.173918,2024-09-23 07:18:00+00:00,rolling_return,False,103.746194
2,2024-09-25 19:38:00+00:00,GRT/USD,573.547803,0.180885,0.173918,2024-09-23 07:18:00+00:00,return,True,103.746194
6,2024-09-25 19:38:00+00:00,GRT/USD,573.547803,0.180885,0.173918,2024-09-23 07:18:00+00:00,coev,True,103.746194
4,2024-09-25 19:38:00+00:00,CRV/USD,338.072563,0.306720,0.295055,2024-09-23 07:18:00+00:00,avg_return,True,103.693616
7,2024-09-25 19:38:00+00:00,USDT/USD,99.756983,0.999863,0.999930,2024-09-23 07:18:00+00:00,coev,False,99.743316
0,2024-09-25 19:38:00+00:00,AAVE/USD,0.586989,166.150000,169.935000,2024-09-23 07:18:00+00:00,rolling_return,True,97.528246
3,2024-09-25 19:38:00+00:00,AAVE/USD,0.586989,166.150000,169.935000,2024-09-23 07:18:00+00:00,return,False,97.528246
5,2024-09-25 19:38:00+00:00,AAVE/USD,0.586989,166.150000,169.935000,2024-09-23 07:18:00+00:00,avg_return,False,97.528246
